# Code Execution with Assistants API

In this notebook we will introduce the concepts of code execution with Code Interpreters, such as Assistants API and Taskweaver. We will experiment with different prompts here with Assistants API, and we will see how we can get these CIs to generate and execute code for us. 


### Python Imports


In [1]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('..\\code')


import os
from dotenv import load_dotenv
load_dotenv()

from IPython.display import display, Markdown, HTML
from PIL import Image
import webbrowser
from doc_utils import *


def show_img(img_path, width = None):
    if width is not None:
        display(HTML(f'<img src="{img_path}" width={width}>'))
    else:
        display(Image.open(img_path))


### Make sure we have the OpenAI Models information

We will need the GPT-4-Turbo for this notebook. 

When running the below cell, the values should reflect the OpenAI reource you have created in the `.env` file.


In [2]:
model_info = {
        'AZURE_OPENAI_RESOURCE': os.environ.get('AZURE_OPENAI_RESOURCE'),
        'AZURE_OPENAI_KEY': os.environ.get('AZURE_OPENAI_KEY'),
        'AZURE_OPENAI_MODEL_VISION': os.environ.get('AZURE_OPENAI_MODEL_VISION'),
        'AZURE_OPENAI_MODEL': os.environ.get('AZURE_OPENAI_MODEL'),
}

oai_client = AzureOpenAI(
    azure_endpoint = OPENAI_API_BASE, 
    api_key= AZURE_OPENAI_KEY,  
    api_version= AZURE_OPENAI_API_VERSION,
)

oai_emb_client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_EMBEDDING_API_BASE, 
    api_key= AZURE_OPENAI_EMBEDDING_MODEL_RESOURCE_KEY,  
    api_version= AZURE_OPENAI_EMBEDDING_MODEL_API_VERSION,
)

model_info

{'AZURE_OPENAI_RESOURCE': 'oai-tst-sweden',
 'AZURE_OPENAI_KEY': '3b32428e172a42329d8b7b1ef042ec1c',
 'AZURE_OPENAI_MODEL_VISION': 'gpt4v',
 'AZURE_OPENAI_MODEL': 'gpt-4'}


## Assistants API

In this section, we will use Assistants API to solve a specific problem. Assistants API can generate and run code in a safe environment to produce a solution.

The first step is to create the Assistant.

In [3]:
name = "Navigator Assistant"
instructions = "You are a Navigator assistant. You are helping me navigate through earth geographies and providing me with accurate GPS coordinates."


assistant, thread = create_assistant(oai_client, name=name, instructions=instructions)    

logc(f'Assistant ID', assistant.id)
logc(f'Thread ID', thread.id)


Assistant ID: asst_KTzEmXgJ1O2sTOurbbz911CS

Thread ID: thread_JYc3yj0AQmjcRVGYE6MWyzox


### Assistant and Thread Retrieval

Between runs, or even if the app or kernel are restarted, the same assistants and threads can be retrieved since these IDs are stored on the cloud.

In [4]:
## My Assistant ID is asst_81XUJpzkZNl3JFCq4ElYlcna

assistant = retrieve_assistant(assistant.id)
logc(f'Assistant ID', assistant.id)


Assistant ID: asst_KTzEmXgJ1O2sTOurbbz911CS


In [26]:
## My Thread ID is thread_3Aa0tZdepeK1v0Y3zcsvRuXq

thread = retrieve_thread(thread.id)
logc(f'Thread ID', thread.id)


Thread ID: thread_JYc3yj0AQmjcRVGYE6MWyzox


### Defining Code

Below we define the code for the assistant to help us with the navigation.

In [ ]:
def ask_assistant(query, assistant, thread, client = oai_client):
    messages = query_assistant(query, assistant, thread, client = oai_client)
    response, files = process_assistants_api_response(messages, client = oai_client)
    print(f"Assistants API generated {len(files)} files for this answer.\n")
    logc(f'Assistant Response', response)
    return response, files

#### Asking the Assistant

Asking the Assistant some geography questions to showcase capabilities.

In [6]:
query = "Where is the Eiffel Tower located? Give me the precise GPS coordinates."
response, files = ask_assistant(query, assistant, thread, client = oai_client)

Assistants API generated 0 files for this answer.


Assistant Response: The Eiffel Tower is located in Paris, France. The precise GPS coordinates are approximately 48.8584° N latitude and 2.2945° E longitude.


In [7]:
query = "Where is the Burj Khalifa located? Give me the precise GPS coordinates."
response, files = ask_assistant(query, assistant, thread, client = oai_client)

Assistants API generated 0 files for this answer.


Assistant Response: The Burj Khalifa is located in Dubai, United Arab Emirates. The precise GPS coordinates are approximately 25.1972° N, 55.2744° E.


In [9]:
query = "Where is Mount Kilimanjaro? Give me the precise GPS coordinates."
response, files = ask_assistant(query, assistant, thread, client = oai_client)

Assistants API generated 0 files for this answer.


Assistant Response: Mount Kilimanjaro is located in Tanzania, East Africa. The precise GPS coordinates for its highest peak, Uhuru Peak, are approximately 3.0758° S latitude and 37.3533° E longitude.


In [21]:
query = "Give me the exact distance betwen the landmarks. If the question requires calculations, make sure to break down your methodology step by step in the final answer, walk the user through your process, and show intermediate calculation results. Please provide as a file the code that you have generated and executed so that the user can check the validity of the code."

response, files = ask_assistant(query, assistant, thread, client = oai_client)


Assistants API generated 4 files for this answer.


Assistant Response: The exact distances between the landmarks are as follows:

- Distance between the Eiffel Tower and Burj Khalifa: approximately 5258.34 km
- Distance between the Burj Khalifa and Mount Kilimanjaro: approximately 3678.40 km
- Distance between Mount Kilimanjaro and the Eiffel Tower: approximately 6675.99 km

To verify the validity of the calculations, you can download the Python script file that contains the code used to perform these calculations. Here is the link to download the code file:

[Download the distance calculations code (Python script)](../downloads\distance_calculations.py)


In [24]:
for f in files:
    if (f['type'] == 'file') and (f['asset'].endswith('.py')):
        display(Markdown("```python\n" + read_asset_file(f['asset'])[0] + "\n```"))

```python

import geopy.distance

# Coordinates for the landmarks
eiffel_tower_coords = (48.8584, 2.2945)
burj_khalifa_coords = (25.1972, 55.2744)
kilimanjaro_coords = (-3.0758, 37.3533)

# Function to calculate the great-circle distance between two points
def calculate_distance(coord1, coord2):
    return geopy.distance.distance(coord1, coord2).kilometers

# Calculate distances between the landmarks
distance_eiffel_burj = calculate_distance(eiffel_tower_coords, burj_khalifa_coords)
distance_burj_kilimanjaro = calculate_distance(burj_khalifa_coords, kilimanjaro_coords)
distance_kilimanjaro_eiffel = calculate_distance(kilimanjaro_coords, eiffel_tower_coords)

# The calculated distances (in kilometers)
print(f"Distance between Eiffel Tower and Burj Khalifa: {distance_eiffel_burj:.2f} km")
print(f"Distance between Burj Khalifa and Mount Kilimanjaro: {distance_burj_kilimanjaro:.2f} km")
print(f"Distance between Mount Kilimanjaro and Eiffel Tower: {distance_kilimanjaro_eiffel:.2f} km")

```

In [20]:
query = "Please draw a map with the 3 landmarks on it. And mark the distance between them with links."
response, files = ask_assistant(query, assistant, thread, client = oai_client)


for f in files:
    if (f['type'] == 'file') and (f['asset'].endswith('.html')):
        webbrowser.open(f['asset'])

Assistants API generated 3 files for this answer.


Assistant Response: I have drawn the map again with the Eiffel Tower, Burj Khalifa, and Mount Kilimanjaro marked on it. This time, I included labels on the lines between the landmarks that indicate the distances between them. You can view and interact with the map by downloading it from the link provided below:

[Download the map with distances between landmarks](../downloads\map_with_distances.html)
